In [1]:
#typical imports
import random
from tqdm import tqdm
import numpy as np
import os

#other imports
!pip install top2vec -q
from top2vec import Top2Vec
# import fasttext


     |████████████████████████████████| 6.4MB 11.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 1.2MB 49.4MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


FileNotFoundError: ignored

In [ ]:
!pip uninstall numpy -q
!pip install numpy==1.19.5 -q

In [ ]:
#train fastText

#cbow model :
ft_embeddings = fasttext.train_unsupervised('booking_sentences_processed.txt', model='cbow')

# Skipgram model :
# ft_embeddings = fasttext.train_unsupervised('fastText_training_data.txt', model='skipgram', verbose=True)


In [ ]:
ft_embeddings.save_model('fast_text_embeddings.bin')

In [2]:
#train doc2vec
!wget https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
documents = []
with open('booking_sentences_processed.txt', 'r') as f:
    documents = f.readlines()
# doc2vec = Top2Vec(documents, embedding_model='doc2vec', workers=8, document_ids=list(range(0,len(documents))), speed="deep-learn", keep_documents=False)
# doc2vec.save('doc2vec_top2vec.bin')

--2021-06-09 21:42:59--  https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8092842 (7.7M) [text/plain]
Saving to: ‘booking_sentences_processed.txt’

booking_sentences_p 100%[===================>]   7.72M  48.9MB/s    in 0.2s    

2021-06-09 21:43:00 (48.9 MB/s) - ‘booking_sentences_processed.txt’ saved [8092842/8092842]



In [ ]:
#Build model with USE
use = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
use.save('use_top2vec.bin')

In [9]:
#load sts dataset for evaluation
# with open('sts-mt.csv', 'r') as f:
#   sts_lines = f.readlines() 

# sts_df = []
# for l in sts_lines:
#   sts_df.append(l[:-1].split('\t'))

# sentences1, sentences2, scores = [], [], []
# for l in sts_df:
#   scores.append(l[3])
#   sentences1.append(l[4])
#   sentences2.append(l[5])
# #scores are from 0 to 5. So the following line normalizes the score
# scores = [float(s)/5.0 for s in scores]
documents = [d[:-1] for d in documents] #remove \n character
import random
sentences = random.sample(documents, 1000)
documents = list(set(documents) - set(sentences))
scores = [0.99 for i in range(len(sentences))]
print(len(scores), len(sentences))

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f255500ce90>> (for pre_run_cell):


Exception: ignored

1000 1000
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f255500ce90>> (for post_run_cell):


Exception: ignored

In [ ]:
#Build model with SBert
from sentence_transformers import SentenceTransformer
#download and save the sbert model
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')
sbert.save("./stsb-mpnet-base-v2.pt")
#load and use SBert model
sbert = Top2Vec(documents, embedding_model_path='./stsb-mpnet-base-v2.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
sbert.save('sbert_top2vec.bin')

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
#Set up weights and biases to visualize the training of TSDAE
!pip install wandb -qqq
import wandb
wandb.login()
wandb.init(
  project='topic_modelling_booking_reviews',
  config={
      'pretrained_model':'bert-base-uncased',
      'model': 'TSDAE',
      'evaluation_metric': 'reconstruction loss',
      'dataset':'booking.com reviews (randomly sampled 100k)',
      'weight_decay':0,
      'scheduler':'constantlr',
      'optimizer_params':{'lr': 3e-5},
  }
)

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 174kB 47.4MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akashjorss (use `wandb login --relogin` to force relogin)


In [10]:
#train TSDAE
!pip install sentence_transformers -q
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

from sentence_transformers import evaluation
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences, sentences, scores)

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased' #can we use stsb-mpnet-base-v2?
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=False, pooling_mode_cls_token=True, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

model.to('cuda')

# Define a list with sentences (1k - 100k sentences)
train_sentences = documents

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
# train_dataset.to('cuda')

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

score_list = []
steps_list = []
import matplotlib.pyplot as plt

#callback function
def call_back(score, epoch, steps):
  wandb.log({"score":score, "steps":steps, "epoch": epoch, "num_sentences":steps*8, "loss":1-score})
  print("steps:", steps, "score:", score, "loss:", 1-score)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True,
    save_best_model=True,
    checkpoint_path = './tsdae_checkpoints/',
    checkpoint_save_steps = 500, 
    checkpoint_save_total_limit = 2,
    evaluation_steps=100,
    callback=call_back,
    evaluator=evaluator

)
wandb.finish()
# model.save('tsdae-model.pt')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f255500ce90>> (for pre_run_cell):


Exception: ignored

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  much help in the complex case.
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f255500ce90>> (for post_run_cell):


Exception: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#build top2vec model using tsdae
tsdae = Top2Vec(documents, embedding_model_path='./tsdae.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
tsdae.save('sbert_top2vec.bin')

In [ ]:
import torch
torch.save(model, "tsdae.pt")

In [ ]:
"""
@article{wang-2021-TSDAE,
    title = "TSDAE: Using Transformer-based Sequential Denoising Auto-Encoderfor Unsupervised Sentence Embedding Learning",
    author = "Wang, Kexin and Reimers, Nils and  Gurevych, Iryna", 
    journal= "arXiv preprint arXiv:2104.06979",
    month = "4",
    year = "2021",
    url = "https://arxiv.org/abs/2104.06979",
}
"""


In [ ]:
"""The performance of TSDAE reduced over time on STS possibly because of 
we did lemm., stop word removal, etc. and the sentence structure doesn't match anymore. 
The only way I can think of right now is to apply the text processing steps to STS data set and then evaluate.""" 